# Our Model

In [4]:
import os
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Dropout, Conv2D, SeparableConv2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Reshape, Conv2D, Multiply, LayerNormalization, MultiHeadAttention, Add, Flatten
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import ModelCheckpoint

data_path = "Bijie_landslide_dataset/Bijie-landslide-dataset"
img_size = (224, 224)
batch_size = 8
num_classes = 2


def spatial_attention_module(input_feature):
    # 使用一个1x1的卷积层来生成注意力图
    attention = Conv2D(1, (1, 1), activation='sigmoid')(input_feature)
    # 将注意力图乘以原特征图来增强重要特征
    enhanced_feature = Multiply()([input_feature, attention])
    return enhanced_feature

def build_image_model(input_shape):
    img_input = Input(shape=input_shape, name='image_input')
    img_base_model = EfficientNetB7(include_top=False, input_tensor=img_input, weights='imagenet')
    img_features = img_base_model.output
    img_features = spatial_attention_module(img_features)
    img_features = GlobalAveragePooling2D()(img_features)
    return Model(inputs=img_input, outputs=img_features, name='ImageModel')

def load_images(folder, filenames, color_mode='rgb'):
    images = []
    for filename in filenames:
        if filename.startswith('.'):  # 过滤掉隐藏文件
            continue
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=img_size, color_mode=color_mode)
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)


def transformer_block(x, num_heads=4):
    # Layer Normalization
    x_norm = LayerNormalization()(x)
    # Multi-head self-attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=64)(x_norm, x_norm)
    # Skip connection
    x = Add()([x, attn_output])
    # Another layer normalization
    x_norm = LayerNormalization()(x)
    # Feed-forward network
    ff_output = Dense(units=512, activation='relu')(x_norm)
    # Final skip connection
    output = Add()([x, ff_output])
    return output

def prepare_dataset(data_path):
    landslide_path = os.path.join(data_path, "landslide")
    non_landslide_path = os.path.join(data_path, "non-landslide")
    
    landslide_filenames = [filename for filename in os.listdir(os.path.join(landslide_path, "image")) if not filename.startswith('.')]
    non_landslide_filenames = [filename for filename in os.listdir(os.path.join(non_landslide_path, "image")) if not filename.startswith('.')]

    landslide_images = load_images(os.path.join(landslide_path, "image"), landslide_filenames)
    landslide_dems = load_images(os.path.join(landslide_path, "dem"), landslide_filenames, color_mode='grayscale')
    non_landslide_images = load_images(os.path.join(non_landslide_path, "image"), non_landslide_filenames)
    non_landslide_dems = load_images(os.path.join(non_landslide_path, "dem"), non_landslide_filenames, color_mode='grayscale')

    landslide_labels = np.ones(len(landslide_filenames), dtype=int)
    non_landslide_labels = np.zeros(len(non_landslide_filenames), dtype=int)

    images = np.concatenate([landslide_images, non_landslide_images], axis=0)
    dems = np.concatenate([landslide_dems, non_landslide_dems], axis=0)
    labels = np.concatenate([landslide_labels, non_landslide_labels], axis=0)

    return images, dems, labels


images, dems, labels = prepare_dataset(data_path)
labels = to_categorical(labels, num_classes=num_classes)

x_img_train, x_img_val, x_dem_train, x_dem_val, y_train, y_val = train_test_split(images, dems, labels, test_size=0.2, random_state=42)

def build_dem_cnn_model(input_shape):
    inputs = Input(shape=input_shape, name='dem_input')
    x = SeparableConv2D(128, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SeparableConv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, -1))(x)
    x = transformer_block(x)
    x = Flatten()(x)
    return Model(inputs=inputs, outputs=x, name='Advanced_DEM_CNN')


def build_vae_feature_merger(input_dim):
    input_layer = Input(shape=(input_dim,))
    x = Dense(128, activation='relu')(input_layer)
    z_mean = Dense(64)(x)
    z_log_var = Dense(64)(x)
    
    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    z = Lambda(sampling)([z_mean, z_log_var])
    encoder = Model(input_layer, [z_mean, z_log_var, z], name='vae_merger')
    return encoder



# Build the dual input model
def build_dual_input_model(input_shape):
    img_model = build_image_model(input_shape)
    img_features = img_model.output

    dem_input_shape = (img_size[0], img_size[1], 1)  
    dem_model = build_dem_cnn_model(dem_input_shape)
    dem_features = dem_model.output

    merged_features = Concatenate()([img_features, dem_features])
    vae_encoder = build_vae_feature_merger(merged_features.shape[1])
    z_mean, z_log_var, z = vae_encoder(merged_features)

    x = Dense(256, activation='relu')(z)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[img_model.input, dem_model.input], outputs=output, name='DualInputModel')
    return model

# Compile the model
model = build_dual_input_model(input_shape=(224, 224, 3))
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define the checkpoint filepath
checkpoint_filepath = 'best_model.h5'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

# Train the model with the ModelCheckpoint callback
history = model.fit(
    [x_img_train, x_dem_train],
    y_train,
    validation_data=([x_img_val, x_dem_val], y_val),
    batch_size=batch_size,
    epochs=20,
    callbacks=[checkpoint_callback]  # Pass the ModelCheckpoint callback here
)

# Load the best weights
model.load_weights(checkpoint_filepath)

# Predict using the best weights
y_pred_probs = model.predict([x_img_val, x_dem_val])
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['non-landslide', 'landslide']))
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Epoch 1/20
278/278 [==============================] - ETA: 0s - loss: 0.3974 - accuracy: 0.8093
Epoch 1: val_accuracy improved from -inf to 0.94234, saving model to best_model.h5
278/278 [==============================] - 110s 314ms/step - loss: 0.3974 - accuracy: 0.8093 - val_loss: 0.1259 - val_accuracy: 0.9423
Epoch 2/20
278/278 [==============================] - ETA: 0s - loss: 0.1503 - accuracy: 0.9432
Epoch 2: val_accuracy improved from 0.94234 to 0.97477, saving model to best_model.h5
278/278 [==============================] - 84s 304ms/step - loss: 0.1503 - accuracy: 0.9432 - val_loss: 0.0757 - val_accuracy: 0.9748
Epoch 3/20
278/278 [==============================] - ETA: 0s - loss: 0.0839 - accuracy: 0.9739
Epoch 3: val_accuracy did not improve from 0.97477
278/278 [==============================] - 83s 297ms/step - loss: 0.0839 - accuracy: 0.9739 - val_loss: 0.0904 - val_accuracy: 0.9694
Epoch 4/20
278/278 [==============================] - ETA: 0s - loss: 0.0467 - accuracy: 

## Other Model

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用ResNet50）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = ResNet50(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用简单的CNN模型）
input_dem = Input(shape=(image_height, image_width, 3))
dem_conv1 = Conv2D(32, kernel_size=(3, 3), activation='relu')(input_dem)
dem_pool1 = MaxPooling2D(pool_size=(2, 2))(dem_conv1)
dem_conv2 = Conv2D(64, kernel_size=(3, 3), activation='relu')(dem_pool1)
dem_pool2 = MaxPooling2D(pool_size=(2, 2))(dem_conv2)
dem_flat = Flatten()(dem_pool2)
dem_output = Dense(256, activation='relu')(dem_flat)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)




Epoch 1/50
56/56 [==============================] - 12s 107ms/step - loss: 37.7402 - accuracy: 0.7813 - val_loss: 248.1522 - val_accuracy: 0.7523
Epoch 2/50
56/56 [==============================] - 5s 81ms/step - loss: 0.2899 - accuracy: 0.8912 - val_loss: 1.1469 - val_accuracy: 0.8378
Epoch 3/50
56/56 [==============================] - 5s 95ms/step - loss: 0.2820 - accuracy: 0.9250 - val_loss: 0.8555 - val_accuracy: 0.8221
Epoch 4/50
56/56 [==============================] - 5s 87ms/step - loss: 0.2733 - accuracy: 0.9205 - val_loss: 0.2301 - val_accuracy: 0.9099
Epoch 5/50
56/56 [==============================] - 4s 80ms/step - loss: 0.1595 - accuracy: 0.9340 - val_loss: 1.2882 - val_accuracy: 0.8649
Epoch 6/50
56/56 [==============================] - 5s 81ms/step - loss: 0.1184 - accuracy: 0.9583 - val_loss: 0.1512 - val_accuracy: 0.9437
Epoch 7/50
56/56 [==============================] - 5s 82ms/step - loss: 0.1045 - accuracy: 0.9611 - val_loss: 0.1279 - val_accuracy: 0.9414
Epoch 8/

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionV3
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用VGG16）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = VGG16(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用InceptionV3）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 18s 181ms/step - loss: 1.4719 - accuracy: 0.6815 - val_loss: 293.0383 - val_accuracy: 0.7523
Epoch 2/50
56/56 [==============================] - 7s 130ms/step - loss: 0.5777 - accuracy: 0.7142 - val_loss: 0.6067 - val_accuracy: 0.7523
Epoch 3/50
56/56 [==============================] - 7s 133ms/step - loss: 0.5755 - accuracy: 0.7159 - val_loss: 0.5171 - val_accuracy: 0.7523
Epoch 4/50
56/56 [==============================] - 7s 123ms/step - loss: 0.5470 - accuracy: 0.7193 - val_loss: 0.5026 - val_accuracy: 0.7590
Epoch 5/50
56/56 [==============================] - 7s 124ms/step - loss: 0.5639 - accuracy: 0.7153 - val_loss: 0.5108 - val_accuracy: 0.7523
Epoch 6/50
56/56 [==============================] - 8s 139ms/step - loss: 0.5569 - accuracy: 0.7142 - val_loss: 0.4869 - val_accuracy: 0.7635
Epoch 7/50
56/56 [==============================] - 7s 125ms/step - loss: 0.5540 - accuracy: 0.7114 - val_loss: 0.4419 - val_accuracy: 0.7725
Epo

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用ResNet50）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = ResNet50(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-13 11:35:22.958020: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 11:35:36.746160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 11:35:37.181305: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:1b:00.0, compute capability: 8.9


Epoch 1/50


2024-05-13 11:35:51.244375: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-13 11:35:51.934578: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-13 11:35:51.934595: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-13 11:35:51.934666: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-13 11:35:52.720975: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


56/56 [==============================] - 23s 195ms/step - loss: 0.4982 - accuracy: 0.8106 - val_loss: 134.5846 - val_accuracy: 0.7365
Epoch 2/50
56/56 [==============================] - 10s 183ms/step - loss: 0.2596 - accuracy: 0.9115 - val_loss: 47.5036 - val_accuracy: 0.7230
Epoch 3/50
56/56 [==============================] - 10s 180ms/step - loss: 0.1773 - accuracy: 0.9436 - val_loss: 2.3851 - val_accuracy: 0.7545
Epoch 4/50
56/56 [==============================] - 9s 159ms/step - loss: 0.1328 - accuracy: 0.9583 - val_loss: 0.6486 - val_accuracy: 0.8806
Epoch 5/50
56/56 [==============================] - 9s 152ms/step - loss: 0.1449 - accuracy: 0.9538 - val_loss: 8.0047 - val_accuracy: 0.7635
Epoch 6/50
56/56 [==============================] - 9s 161ms/step - loss: 0.1155 - accuracy: 0.9583 - val_loss: 0.8783 - val_accuracy: 0.8919
Epoch 7/50
56/56 [==============================] - 10s 175ms/step - loss: 0.0689 - accuracy: 0.9752 - val_loss: 0.6176 - val_accuracy: 0.8514
Epoch 8/50

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-13 11:59:58.884454: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2024-05-13 12:00:12.912068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 12:00:13.332656: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:39:00.0, compute capability: 8.9


Epoch 1/50


2024-05-13 12:00:25.916309: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-13 12:00:26.561397: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-13 12:00:26.561414: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-13 12:00:26.561497: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-13 12:00:27.376543: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


56/56 [==============================] - 27s 287ms/step - loss: 0.2652 - accuracy: 0.8963 - val_loss: 6.2467 - val_accuracy: 0.7725
Epoch 2/50
56/56 [==============================] - 15s 260ms/step - loss: 0.1569 - accuracy: 0.9560 - val_loss: 10.0856 - val_accuracy: 0.2477
Epoch 3/50
56/56 [==============================] - 14s 258ms/step - loss: 0.0890 - accuracy: 0.9701 - val_loss: 8.4199 - val_accuracy: 0.2477
Epoch 4/50
56/56 [==============================] - 14s 241ms/step - loss: 0.0904 - accuracy: 0.9690 - val_loss: 9.6985 - val_accuracy: 0.3131
Epoch 5/50
56/56 [==============================] - 14s 244ms/step - loss: 0.0436 - accuracy: 0.9865 - val_loss: 6.4101 - val_accuracy: 0.4482
Epoch 6/50
56/56 [==============================] - 14s 246ms/step - loss: 0.0525 - accuracy: 0.9837 - val_loss: 7.8207 - val_accuracy: 0.3311
Epoch 7/50
56/56 [==============================] - 13s 240ms/step - loss: 0.0640 - accuracy: 0.9769 - val_loss: 15.1759 - val_accuracy: 0.2477
Epoch 8/

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet, DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNet）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNet(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用DenseNet121）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 22s 199ms/step - loss: 0.2480 - accuracy: 0.9092 - val_loss: 0.5549 - val_accuracy: 0.9099
Epoch 2/50
56/56 [==============================] - 9s 169ms/step - loss: 0.0831 - accuracy: 0.9729 - val_loss: 0.2890 - val_accuracy: 0.9324
Epoch 3/50
56/56 [==============================] - 10s 184ms/step - loss: 0.0836 - accuracy: 0.9735 - val_loss: 0.1958 - val_accuracy: 0.9324
Epoch 4/50
56/56 [==============================] - 10s 172ms/step - loss: 0.0694 - accuracy: 0.9780 - val_loss: 0.0429 - val_accuracy: 0.9865
Epoch 5/50
56/56 [==============================] - 8s 152ms/step - loss: 0.0486 - accuracy: 0.9814 - val_loss: 0.1374 - val_accuracy: 0.9640
Epoch 6/50
56/56 [==============================] - 9s 152ms/step - loss: 0.0465 - accuracy: 0.9882 - val_loss: 0.0750 - val_accuracy: 0.9752
Epoch 7/50
56/56 [==============================] - 9s 153ms/step - loss: 0.0301 - accuracy: 0.9870 - val_loss: 0.1061 - val_accuracy: 0.9730
Epo

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用InceptionResNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
dem_features = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)(input_dem)
dem_features = GlobalAveragePooling2D()(dem_features)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


219055592/219055592 [==============================] - 8s 0us/step
Epoch 1/50
56/56 [==============================] - 35s 283ms/step - loss: 0.2934 - accuracy: 0.9014 - val_loss: 7.8982 - val_accuracy: 0.2477
Epoch 2/50
56/56 [==============================] - 12s 222ms/step - loss: 0.1427 - accuracy: 0.9436 - val_loss: 8.9434 - val_accuracy: 0.2477
Epoch 3/50
56/56 [==============================] - 12s 216ms/step - loss: 0.1165 - accuracy: 0.9628 - val_loss: 10.6325 - val_accuracy: 0.2477
Epoch 4/50
56/56 [==============================] - 12s 217ms/step - loss: 0.1006 - accuracy: 0.9656 - val_loss: 7.6408 - val_accuracy: 0.2477
Epoch 5/50
56/56 [==============================] - 12s 217ms/step - loss: 0.0963 - accuracy: 0.9713 - val_loss: 8.0092 - val_accuracy: 0.3896
Epoch 6/50
56/56 [==============================] - 12s 213ms/step - loss: 0.0796 - accuracy: 0.9791 - val_loss: 7.0513 - val_accuracy: 0.4324
Epoch 7/50
56/56 [==============================] - 12s 214ms/step - loss:

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 19s 174ms/step - loss: 0.6861 - accuracy: 0.8608 - val_loss: 19.5984 - val_accuracy: 0.2477
Epoch 2/50
56/56 [==============================] - 8s 145ms/step - loss: 0.1674 - accuracy: 0.9363 - val_loss: 0.3421 - val_accuracy: 0.9099
Epoch 3/50
56/56 [==============================] - 8s 143ms/step - loss: 0.1084 - accuracy: 0.9566 - val_loss: 0.5393 - val_accuracy: 0.8694
Epoch 4/50
56/56 [==============================] - 8s 144ms/step - loss: 0.1126 - accuracy: 0.9526 - val_loss: 0.1573 - val_accuracy: 0.9414
Epoch 5/50
56/56 [==============================] - 8s 144ms/step - loss: 0.1094 - accuracy: 0.9594 - val_loss: 0.2033 - val_accuracy: 0.9167
Epoch 6/50
56/56 [==============================] - 8s 144ms/step - loss: 0.1032 - accuracy: 0.9617 - val_loss: 0.9023 - val_accuracy: 0.9189
Epoch 7/50
56/56 [==============================] - 8s 144ms/step - loss: 0.0926 - accuracy: 0.9684 - val_loss: 8.7821 - val_accuracy: 0.8086
Epoc

In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 20s 175ms/step - loss: 0.7515 - accuracy: 0.8681 - val_loss: 175.0231 - val_accuracy: 0.7995
Epoch 2/50
56/56 [==============================] - 8s 145ms/step - loss: 0.1565 - accuracy: 0.9425 - val_loss: 2.4889 - val_accuracy: 0.6059
Epoch 3/50
56/56 [==============================] - 8s 145ms/step - loss: 0.1522 - accuracy: 0.9521 - val_loss: 0.6890 - val_accuracy: 0.8581
Epoch 4/50
56/56 [==============================] - 8s 145ms/step - loss: 0.1372 - accuracy: 0.9521 - val_loss: 3.1664 - val_accuracy: 0.6869
Epoch 5/50
56/56 [==============================] - 8s 145ms/step - loss: 0.0896 - accuracy: 0.9667 - val_loss: 0.3454 - val_accuracy: 0.8919
Epoch 6/50
56/56 [==============================] - 8s 144ms/step - loss: 0.1302 - accuracy: 0.9521 - val_loss: 0.7794 - val_accuracy: 0.8446
Epoch 7/50
56/56 [==============================] - 8s 144ms/step - loss: 0.0733 - accuracy: 0.9735 - val_loss: 0.5532 - val_accuracy: 0.8378
Epo

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用InceptionV3）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 19s 178ms/step - loss: 0.2900 - accuracy: 0.8895 - val_loss: 143.5713 - val_accuracy: 0.2477
Epoch 2/50
56/56 [==============================] - 8s 146ms/step - loss: 0.1653 - accuracy: 0.9425 - val_loss: 47.4714 - val_accuracy: 0.7545
Epoch 3/50
56/56 [==============================] - 8s 149ms/step - loss: 0.0984 - accuracy: 0.9679 - val_loss: 1.7920 - val_accuracy: 0.6036
Epoch 4/50
56/56 [==============================] - 8s 146ms/step - loss: 0.0990 - accuracy: 0.9673 - val_loss: 2.1073 - val_accuracy: 0.7523
Epoch 5/50
56/56 [==============================] - 8s 147ms/step - loss: 0.0952 - accuracy: 0.9611 - val_loss: 2.0565 - val_accuracy: 0.7793
Epoch 6/50
56/56 [==============================] - 9s 155ms/step - loss: 0.0748 - accuracy: 0.9735 - val_loss: 1.7113 - val_accuracy: 0.8153
Epoch 7/50
56/56 [==============================] - 9s 169ms/step - loss: 0.0460 - accuracy: 0.9859 - val_loss: 0.9559 - val_accuracy: 0.8874
Ep

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用DenseNet121）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用VGG16）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = VGG16(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 19s 176ms/step - loss: 6.8157 - accuracy: 0.8760 - val_loss: 9.2504 - val_accuracy: 0.7410
Epoch 2/50
56/56 [==============================] - 8s 145ms/step - loss: 0.1531 - accuracy: 0.9436 - val_loss: 0.9907 - val_accuracy: 0.8851
Epoch 3/50
56/56 [==============================] - 8s 146ms/step - loss: 0.1869 - accuracy: 0.9340 - val_loss: 0.2146 - val_accuracy: 0.9302
Epoch 4/50
56/56 [==============================] - 8s 146ms/step - loss: 0.1234 - accuracy: 0.9549 - val_loss: 0.2582 - val_accuracy: 0.9009
Epoch 5/50
56/56 [==============================] - 8s 146ms/step - loss: 0.1059 - accuracy: 0.9600 - val_loss: 0.4798 - val_accuracy: 0.9009
Epoch 6/50
56/56 [==============================] - 9s 154ms/step - loss: 0.0717 - accuracy: 0.9729 - val_loss: 1.0122 - val_accuracy: 0.7410
Epoch 7/50
56/56 [==============================] - 9s 162ms/step - loss: 0.0957 - accuracy: 0.9713 - val_loss: 0.2300 - val_accuracy: 0.9324
Epoch

In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, MobileNetV2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用VGG19）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = VGG19(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用MobileNetV2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


80134624/80134624 [==============================] - 3s 0us/step
Epoch 1/50
56/56 [==============================] - 19s 197ms/step - loss: 7.8709 - accuracy: 0.6714 - val_loss: 2.1752 - val_accuracy: 0.7523
Epoch 2/50
56/56 [==============================] - 10s 174ms/step - loss: 0.6076 - accuracy: 0.7153 - val_loss: 1.6520 - val_accuracy: 0.7523
Epoch 3/50
56/56 [==============================] - 10s 175ms/step - loss: 0.5790 - accuracy: 0.7170 - val_loss: 0.7364 - val_accuracy: 0.7523
Epoch 4/50
56/56 [==============================] - 10s 174ms/step - loss: 0.5680 - accuracy: 0.7198 - val_loss: 0.9486 - val_accuracy: 0.7523
Epoch 5/50
56/56 [==============================] - 10s 175ms/step - loss: 0.5645 - accuracy: 0.7131 - val_loss: 0.7159 - val_accuracy: 0.7523
Epoch 6/50
56/56 [==============================] - 10s 174ms/step - loss: 0.5726 - accuracy: 0.7204 - val_loss: 0.5931 - val_accuracy: 0.7523
Epoch 7/50
56/56 [==============================] - 10s 174ms/step - loss: 0.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3, DenseNet121
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用InceptionV3）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = InceptionV3(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用DenseNet121）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = DenseNet121(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


2024-05-13 13:34:26.356605: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 13:34:39.622451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 13:34:40.061789: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:39:00.0, compute capability: 8.9


Epoch 1/50


2024-05-13 13:35:01.083177: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2024-05-13 13:35:01.742881: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-05-13 13:35:01.742895: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2024-05-13 13:35:01.742940: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-05-13 13:35:02.814401: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


56/56 [==============================] - 31s 224ms/step - loss: 0.3265 - accuracy: 0.8822 - val_loss: 3.5029 - val_accuracy: 0.7568
Epoch 2/50
56/56 [==============================] - 8s 144ms/step - loss: 0.1473 - accuracy: 0.9498 - val_loss: 5.2990 - val_accuracy: 0.8333
Epoch 3/50
56/56 [==============================] - 8s 142ms/step - loss: 0.0979 - accuracy: 0.9634 - val_loss: 0.5697 - val_accuracy: 0.8446
Epoch 4/50
56/56 [==============================] - 9s 157ms/step - loss: 0.0992 - accuracy: 0.9628 - val_loss: 0.5546 - val_accuracy: 0.8401
Epoch 5/50
56/56 [==============================] - 9s 163ms/step - loss: 0.0789 - accuracy: 0.9707 - val_loss: 0.2883 - val_accuracy: 0.9279
Epoch 6/50
56/56 [==============================] - 9s 160ms/step - loss: 0.0644 - accuracy: 0.9769 - val_loss: 0.2000 - val_accuracy: 0.9414
Epoch 7/50
56/56 [==============================] - 8s 143ms/step - loss: 0.1110 - accuracy: 0.9605 - val_loss: 0.2846 - val_accuracy: 0.8941
Epoch 8/50
56/56

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用MobileNetV2）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Epoch 1/50
56/56 [==============================] - 26s 286ms/step - loss: 0.2662 - accuracy: 0.8952 - val_loss: 6.6084 - val_accuracy: 0.2680
Epoch 2/50
56/56 [==============================] - 14s 247ms/step - loss: 0.1125 - accuracy: 0.9605 - val_loss: 10.7866 - val_accuracy: 0.2477
Epoch 3/50
56/56 [==============================] - 14s 247ms/step - loss: 0.0826 - accuracy: 0.9741 - val_loss: 10.5499 - val_accuracy: 0.2477
Epoch 4/50
56/56 [==============================] - 14s 249ms/step - loss: 0.0893 - accuracy: 0.9718 - val_loss: 10.9711 - val_accuracy: 0.2477
Epoch 5/50
56/56 [==============================] - 13s 235ms/step - loss: 0.1005 - accuracy: 0.9656 - val_loss: 8.3066 - val_accuracy: 0.4144
Epoch 6/50
56/56 [==============================] - 14s 246ms/step - loss: 0.0650 - accuracy: 0.9808 - val_loss: 9.6522 - val_accuracy: 0.2568
Epoch 7/50
56/56 [==============================] - 14s 242ms/step - loss: 0.0336 - accuracy: 0.9893 - val_loss: 16.1590 - val_accuracy: 0.

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet152V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用EfficientNetB0）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用ResNet152V2）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = ResNet152V2(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


234545216/234545216 [==============================] - 9s 0us/step
Epoch 1/50
56/56 [==============================] - 36s 303ms/step - loss: 0.2331 - accuracy: 0.9166 - val_loss: 0.4108 - val_accuracy: 0.9099
Epoch 2/50
56/56 [==============================] - 13s 233ms/step - loss: 0.0940 - accuracy: 0.9707 - val_loss: 0.1393 - val_accuracy: 0.9752
Epoch 3/50
56/56 [==============================] - 14s 248ms/step - loss: 0.0415 - accuracy: 0.9848 - val_loss: 0.1198 - val_accuracy: 0.9685
Epoch 4/50
56/56 [==============================] - 14s 254ms/step - loss: 0.0586 - accuracy: 0.9769 - val_loss: 0.0478 - val_accuracy: 0.9865
Epoch 5/50
56/56 [==============================] - 14s 255ms/step - loss: 0.0384 - accuracy: 0.9870 - val_loss: 0.1212 - val_accuracy: 0.9730
Epoch 6/50
56/56 [==============================] - 13s 239ms/step - loss: 0.0625 - accuracy: 0.9786 - val_loss: 0.0634 - val_accuracy: 0.9662
Epoch 7/50
56/56 [==============================] - 13s 238ms/step - loss: 

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import NASNetMobile, Xception
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 数据集路径
data_dir = "Bijie_landslide_dataset/Bijie-landslide-dataset"

# 定义数据生成器
datagen = ImageDataGenerator(rescale=1./255)

# 加载数据集
batch_size = 32
image_height = 224
image_width = 224

# 加载landslide数据
landslide_dir = os.path.join(data_dir, "landslide")
landslide_dem_dir = os.path.join(landslide_dir, "dem")
landslide_image_dir = os.path.join(landslide_dir, "image")
landslide_mask_dir = os.path.join(landslide_dir, "mask")

# 加载non-landslide数据
non_landslide_dir = os.path.join(data_dir, "non-landslide")
non_landslide_dem_dir = os.path.join(non_landslide_dir, "dem")
non_landslide_image_dir = os.path.join(non_landslide_dir, "image")

# 读取并预处理数据函数
def read_and_preprocess_data(image_dir, dem_dir=None):
    image_paths = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir) if not filename.startswith('.')]
    images = [tf.keras.preprocessing.image.load_img(img_path, target_size=(image_height, image_width)) for img_path in image_paths]
    images = [tf.keras.preprocessing.image.img_to_array(img) for img in images]
    images = np.array(images)

    if dem_dir:
        dem_paths = [os.path.join(dem_dir, filename) for filename in os.listdir(dem_dir) if not filename.startswith('.')]
        dem_images = [tf.keras.preprocessing.image.load_img(dem_path, target_size=(image_height, image_width)) for dem_path in dem_paths]
        dem_images = [tf.keras.preprocessing.image.img_to_array(dem) for dem in dem_images]
        dem_images = np.array(dem_images)
        return images, dem_images
    else:
        return images

# 读取并预处理landslide数据
landslide_images, landslide_dem_images = read_and_preprocess_data(landslide_image_dir, landslide_dem_dir)
landslide_labels = np.ones(len(landslide_images))

# 读取并预处理non-landslide数据
non_landslide_images = read_and_preprocess_data(non_landslide_image_dir)
non_landslide_labels = np.zeros(len(non_landslide_images))

# 合并数据集
images = np.concatenate((landslide_images, non_landslide_images), axis=0)
dem_images = np.concatenate((landslide_dem_images, np.zeros_like(non_landslide_images)), axis=0)  # 对于non-landslide数据，DEM图像全为0
labels = np.concatenate((landslide_labels, non_landslide_labels), axis=0)

# 切分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# 构建图像处理模型（使用NASNetMobile）
input_image = Input(shape=(image_height, image_width, 3))
base_model_image = NASNetMobile(include_top=False, weights='imagenet', input_tensor=input_image)
image_features = GlobalAveragePooling2D()(base_model_image.output)
image_output = Dense(256, activation='relu')(image_features)

# 构建DEM处理模型（使用Xception）
input_dem = Input(shape=(image_height, image_width, 3))
base_model_dem = Xception(include_top=False, weights='imagenet', input_tensor=input_dem)
dem_features = GlobalAveragePooling2D()(base_model_dem.output)
dem_output = Dense(256, activation='relu')(dem_features)

# 将两个模型输出连接
merged = Concatenate()([image_output, dem_output])

# 最终的分类层
output = Dense(1, activation='sigmoid')(merged)

# 定义模型
model = Model(inputs=[input_image, input_dem], outputs=output)

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit([X_train, dem_images[:len(X_train)]], y_train, batch_size=batch_size, epochs=50, validation_split=0.2)

# 评估模型
test_loss, test_acc = model.evaluate([X_test, dem_images[len(X_train):]], y_test, verbose=2)
print('\nTest accuracy:', test_acc)

# 模型预测
y_pred = model.predict([X_test, dem_images[len(X_train):]])
y_pred_classes = np.round(y_pred)

# 输出性能报告结果
report = classification_report(y_test, y_pred_classes, target_names=['non-landslide', 'landslide'])
print("Classification Report:\n", report)

# 输出其他性能指标
accuracy = np.mean(y_test == y_pred_classes)
precision = np.mean(y_pred_classes[y_test == 1] == 1)
recall = np.mean(y_pred_classes[y_test == 1] == y_test[y_test == 1])
f1_score = 2 * precision * recall / (precision + recall)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


19993432/19993432 [==============================] - 2s 0us/step
Epoch 1/50
56/56 [==============================] - 45s 384ms/step - loss: 0.2333 - accuracy: 0.9126 - val_loss: 20.8696 - val_accuracy: 0.2477
Epoch 2/50
56/56 [==============================] - 17s 308ms/step - loss: 0.0786 - accuracy: 0.9763 - val_loss: 899.4100 - val_accuracy: 0.2477
Epoch 3/50
56/56 [==============================] - 17s 310ms/step - loss: 0.0671 - accuracy: 0.9797 - val_loss: 17.3617 - val_accuracy: 0.2500
Epoch 4/50
56/56 [==============================] - 17s 311ms/step - loss: 0.0412 - accuracy: 0.9853 - val_loss: 641.7869 - val_accuracy: 0.2658
Epoch 5/50
56/56 [==============================] - 17s 308ms/step - loss: 0.0547 - accuracy: 0.9842 - val_loss: 18.7561 - val_accuracy: 0.5766
Epoch 6/50
56/56 [==============================] - 17s 311ms/step - loss: 0.0160 - accuracy: 0.9944 - val_loss: 197.5278 - val_accuracy: 0.2725
Epoch 7/50
56/56 [==============================] - 17s 306ms/step -